In [1]:
import requests
import pandas as pd
import numpy as np

import json
import geopandas
from shapely.geometry import Point

import seaborn as sns
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings("ignore")

%matplotlib inline

PROJ: proj_create_from_database: SQLite error on SELECT name, type, coordinate_system_auth_name, coordinate_system_code, datum_auth_name, datum_code, area_of_use_auth_name, area_of_use_code, text_definition, deprecated FROM geodetic_crs WHERE auth_name = ? AND code = ?: no such column: area_of_use_auth_name


In [2]:
def preprocess(df, text_field):
    
    df[text_field] = df[text_field].str.lower()
    
    # удалить ненужные символы и слова
    df[text_field] = df[text_field].str.replace("ё", "e", regex=False)
    df[text_field] = df[text_field].str.replace(",", "", regex=False)
    df[text_field] = df[text_field].str.replace("район", "", regex=False)
    df[text_field] = df[text_field].str.replace("[\"]", "", regex=True)
    df[text_field] = df[text_field].str.replace("\n", "", regex=False)
    df[text_field] = df[text_field].str.replace("поселение", "", regex=False)
    df[text_field] = df[text_field].str.replace("городской округ", "", regex=False)
    df[text_field] = df[text_field].str.replace("административный округ", "", regex=False)
    df[text_field] = df[text_field].str.replace("admarea:", "", regex=False)
    df[text_field] = df[text_field].str.replace("district:", "", regex=False)
    
    # трим нескольких пробелов между словами и удаление крайних пробелов в строке
    df[text_field] = df[text_field].str.replace(r"\s+", " ", regex=True)
    df[text_field] = df[text_field].str.replace(r"^[ \s]+|[ \s]+$", "", regex=True)
    
    return df


def neighborhoods_preprocess(neighborhoods):
    # удалить ненужные столбцы
    columns = ["№", "Флаг", "Герб", "Название района[2]/поселения[3][4]"]
    neighborhoods.drop(columns=columns, inplace=True)


    # переименование столбцов и удаление итоговой строки (вся Москва)
    columns = {
        "Название cоответствующего внутригородского муниципального образования: муниципального округа / поселения / городского округа[5]": "neighborhood",
        "Адми-нистра-тивныйокруг": "administrative_district",
        "Пло-щадь,[6][7]км²": "nhood_area_in_square_km",
        "Насе-ление2022[8],чел.": "nhood_population",
        "Плот-ностьнасе-ления2022,чел. / км²": "nhood_population_density_person_per_square_km",
        "Пло-щадьжилого фонда(01.01.2010)[9],тыс. м²": "housing_area_thousand_square_meters",
        "Жил-площадьначело-века(01.01.2010),м²/чел.": "housing_area_square_meters_per_person",
    }
    neighborhoods.rename(columns=columns, inplace=True)


    # чистка значений
    neighborhoods["nhood_population"] = neighborhoods["nhood_population"].str.replace("↘", "")
    neighborhoods["nhood_population"] = neighborhoods["nhood_population"].str.replace("↗", "")
    neighborhoods["nhood_population"] = neighborhoods["nhood_population"].str.replace("\s", "", regex=True)


    # приведение значений к корректному типу
    types = {
        "nhood_population": "int64",
    }
    neighborhoods = neighborhoods.astype(types)


    # приведение названий округов к единому виду со справочником округов
    districts = {
        "ЮЗАО": "Юго-Западный",
        "СВАО": "Северо-Восточный",
        "ЦАО": "Центральный",
        "САО": "Северный",
        "ЮАО": "Южный",
        "ВАО": "Восточный",
        "ЗАО": "Западный",
        "ЮВАО": "Юго-Восточный",
        "ЗелАО": "Зеленоградский",
        "СЗАО": "Северо-Западный",
        "НАО": "Новомосковский",
        "ТАО": "Троицкий"
    }
    neighborhoods["administrative_district"] = neighborhoods["administrative_district"].map(districts)
    
    neighborhoods["neighborhood"] = neighborhoods["neighborhood"].str.lower()
    neighborhoods["administrative_district"] = neighborhoods["administrative_district"].str.lower()
    
    return neighborhoods

# получить район из геоцентра
def transform_from_geocenter(df, gdf):
    
    def convert(row):
        row_dict = json.loads(row)
        coords = row_dict["coordinates"]
        return Point(coords)

    df = df[df["geodata_center"].notnull()]
    df["geodata_center"] = df["geodata_center"].apply(lambda row: convert(row))
    df = geopandas.GeoDataFrame(df, geometry="geodata_center")
    df.rename(columns={"geodata_center": "geometry"}, inplace=True)

    df = gdf.sjoin(df, how='right')
    df.rename(columns={"okrug_name": "Административный округ", "adm_name": "Район"}, inplace=True)
    
    return df


# подсчет количества объектов по району
def count_objects(df, col_name="objects", set_adress_from_geocenter=False, gdf=None):
    
    if set_adress_from_geocenter:
        df = transform_from_geocenter(df, gdf)
    
    df = preprocess(df, "Район")
    df = preprocess(df, "Административный округ")
    
    grouped_df = df.groupby(["Район"], as_index=False).count()
    grouped_df = grouped_df[["Район", "Административный округ"]]
    grouped_df.rename(columns={"Административный округ": f"{col_name}_cnt"}, inplace=True)
    
    return grouped_df

In [3]:
# геодата от оргов
path_to_data = r"..\data\hackaton_zones\adm2fishnet.csv"
df = pd.read_csv(path_to_data, sep=";")
gdf = geopandas.GeoDataFrame(df)
gdf["geometry"] = geopandas.GeoSeries.from_wkt(df['WKT'])

gdf.drop(columns=['WKT'], inplace=True)
gdf = gdf[(gdf["sub_ter"] != "Московская область")]
gdf = preprocess(gdf, "adm_name")
gdf["boundary"] = gdf.geometry.boundary

In [5]:
# Районы Москвы
url = r"https://ru.wikipedia.org/wiki/%D0%A0%D0%B0%D0%B9%D0%BE%D0%BD%D1%8B_%D0%B8_%D0%BF%D0%BE%D1%81%D0%B5%D0%BB%D0%B5%D0%BD%D0%B8%D1%8F_%D0%9C%D0%BE%D1%81%D0%BA%D0%B2%D1%8B"
r = requests.get(url, verify=False)

neighborhoods = pd.read_html(r.text)[0]
neighborhoods = neighborhoods_preprocess(neighborhoods)
neighborhoods = preprocess(neighborhoods, "neighborhood")

In [74]:
neighborhoods.to_excel(r"..\data\artefacts\neighborhoods.xlsx")

### Возможность получить необходимые товары и услуги в пределах района ###

In [6]:
# спортзалы
dosug_sportrooms = pd.read_excel(r"..\data\data-mos-ru\data-dosug-sportrooms.xlsx", skiprows=1)
dosug_sportrooms = dosug_sportrooms[["Административный округ", "Район", "Адрес", "Форма посещения (платность)"]]

# спорткомплексы
sports_sportcomplexes = pd.read_excel(r"..\data\data-mos-ru\data-sports-sportobjects.xlsx", skiprows=1)
sports_sportcomplexes = sports_sportcomplexes[["Категория", "Услуги", "geodata_center"]]

# фитнес-центры
dosug_fitness = pd.read_excel(r"..\data\data-mos-ru\data-dosug-sportcomplex.xlsx", skiprows=1)
dosug_fitness = dosug_fitness[["Административный округ", "Район", "Адрес", "Форма посещения (платность)"]]


###########################################


# взрослые больнциы
med_adult = pd.read_excel(r"..\data\data-mos-ru\data-med-adult.xlsx", skiprows=1)
med_adult = med_adult[["Категория", "Адреса объекта", "Признак закрытия", "Коечный фонд", "Информация о платных услугах", 
                      "Наличие аптечного киоска", "Наличие станции неотложной помощи", "geodata_center"]]

# взрослые поликлинники
med_adult_policlinic = pd.read_excel(r"..\data\data-mos-ru\data-med-adult-policlinic.xlsx", skiprows=1)
med_adult_policlinic = med_adult_policlinic[["Категория", "Адреса объекта", "Признак закрытия", 
                                           "Информация о платных услугах", "Наличие аптечного киоска", 
                                           "Наличие станции неотложной помощи", "geodata_center"]]

# детские больницы
med_kids = pd.read_excel(r"..\data\data-mos-ru\data-med-kids.xlsx", skiprows=1)
med_kids = med_kids[["Категория", "Адреса объекта", "Признак закрытия", "Коечный фонд", "Наличие аптечного киоска",
                     "Информация о платных услугах", "Наличие станции неотложной помощи", "geodata_center"]]

# детские поликлинники
med_kids_policlinic = pd.read_excel(r"..\data\data-mos-ru\data-med-kids-policlinic.xlsx", skiprows=1)
med_kids_policlinic = med_kids_policlinic[["Категория", "Адреса объекта", "Признак закрытия", 
                                           "Информация о платных услугах", "Наличие аптечного киоска", 
                                           "Наличие станции неотложной помощи", "geodata_center"]]


###########################################

# точки питания
eat_commoneat = pd.read_excel(r"..\data\data-mos-ru\data-eat-commoneat.xlsx", skiprows=1)
eat_commoneat = eat_commoneat[["Является сетевым", "Вид объекта", "Административный округ по адресу", "Район", 
                              "Адрес", "Число посадочных мест", "Долгота в WGS-84", "Широта в WGS-84"]]
eat_commoneat.rename(columns={"Административный округ по адресу": "Административный округ"}, inplace=True)


# оказание услуг
trade_bitovieuslugi = pd.read_excel(r"..\data\data-mos-ru\data-trade-bitovieuslugi.xlsx", skiprows=1)
trade_bitovieuslugi = trade_bitovieuslugi[["Вид объекта бытового обслуживания", "Является сетевым", "Вид услуги", 
                                           "Административный округ", "Район", "Широта в WGS-84", "Долгота в WGS-84"]]

# торговля товарами
trade_stacionarnietochki = pd.read_excel(r"..\data\data-mos-ru\data-trade-stacionarnietochki.xlsx", skiprows=1)
trade_stacionarnietochki = trade_stacionarnietochki[["Вид услуги", "Вид объекта", "Является сетевым", 
                                                    "Административный округ", "Район", "geodata_center"]]

In [7]:
# считаем кол-во объектов по району

sportrooms = count_objects(dosug_sportrooms, "sportrooms")
sportcomplexes = count_objects(sports_sportcomplexes, "sportcomplexes", True, gdf)
fitness = count_objects(dosug_fitness, "fitness")

medicine_adult = count_objects(med_adult, "medicine_adult", True, gdf)
medicine_adult_poli = count_objects(med_adult_policlinic, "medicine_adult_poli", True, gdf)
medicine_kids = count_objects(med_kids, "medicine_kids", True, gdf)
medicine_kids_poli = count_objects(med_kids_policlinic, "medicine_kids_poli", True, gdf)

commoneat = count_objects(eat_commoneat, "commoneat")
services = count_objects(trade_bitovieuslugi, "services")
trades = count_objects(trade_stacionarnietochki, "trades")

In [8]:
# объеденяем все спортобъекты в один фрейм

sports = sportrooms.merge(sportcomplexes, how='outer')
sports = sports.merge(fitness, how='outer')

sports.fillna(0, inplace=True)
sports["sports"] = sports["sportrooms_cnt"] + sports["sportcomplexes_cnt"] + sports["fitness_cnt"]

all_sports = sports[["Район", "sports"]]
all_sports

,Район,sports
0,академический,30.0
1,алексеевский,60.0
2,алтуфьевский,14.0
3,арбат,19.0
4,аэропорт,36.0
...,...,...
145,филевский парк,2.0
146,хорошево-мневники,3.0
147,хорошевский,1.0
148,черемушки,2.0


In [9]:
# объеденяем все больницы и поликлинники в один фрейм, в разбивке по взрослым и детским

adult = medicine_adult.merge(medicine_adult_poli, how='outer')
kids = medicine_kids.merge(medicine_kids_poli, how='outer')
adult_kids = adult.merge(kids, how='outer')

adult_kids.fillna(0, inplace=True)
adult_kids["adult_med"] = adult_kids["medicine_adult_cnt"] + adult_kids["medicine_adult_poli_cnt"]
adult_kids["kids_med"] = adult_kids["medicine_kids_cnt"] + adult_kids["medicine_kids_poli_cnt"]

all_adult_kids = adult_kids[["Район", "adult_med", "kids_med"]]
all_adult_kids

,Район,adult_med,kids_med
0,академический,4.0,2.0
1,внуковское,2.0,1.0
2,вороновское,2.0,1.0
3,даниловский,2.0,1.0
4,донской,4.0,0.0
...,...,...,...
119,нижегородский,0.0,1.0
120,первомайское,0.0,1.0
121,свиблово,0.0,1.0
122,чертаново центральное,0.0,1.0


In [10]:
# объеденяем все бытовые услуги, товары и точки питания в один фрейм

services_trades = services.merge(trades, how='outer')
services_trades_eat = services_trades.merge(commoneat, how='outer')
services_trades_eat.rename(columns={"services_cnt": "services", "trades_cnt": "trades", "commoneat_cnt": "eatpoint"}, 
                           inplace=True)
services_trades_eat

,Район,services,trades,eatpoint
0,академический,181,375,157
1,алексеевский,166,337,157
2,алтуфьевский,43,110,31
3,арбат,138,442,301
4,аэропорт,217,447,204
...,...,...,...,...
141,южное тушино,161,444,121
142,южнопортовый,139,2115,191
143,якиманка,122,218,206
144,ярославский,137,333,102


In [11]:
# объеденяем все фреймы в один

all_services = all_sports.merge(all_adult_kids, how="outer")
all_services = all_services.merge(services_trades_eat, how="outer")
all_services

,Район,sports,adult_med,kids_med,services,trades,eatpoint
0,академический,30.0,4.0,2.0,181.0,375.0,157.0
1,алексеевский,60.0,2.0,2.0,166.0,337.0,157.0
2,алтуфьевский,14.0,0.0,1.0,43.0,110.0,31.0
3,арбат,19.0,1.0,0.0,138.0,442.0,301.0
4,аэропорт,36.0,2.0,0.0,217.0,447.0,204.0
...,...,...,...,...,...,...,...
151,теплый стан,NaN,2.0,2.0,NaN,NaN,NaN
152,филимонковское,NaN,1.0,1.0,2.0,34.0,4.0
153,бирюлeво западное,NaN,NaN,NaN,63.0,183.0,41.0
154,михайлово-ярцевское,NaN,NaN,NaN,9.0,44.0,2.0


In [12]:
# с вики-данных осатвляем нужные и соединяем их с фреймом кол-ва всех услуг

nhoods = neighborhoods[["administrative_district", "neighborhood", "nhood_area_in_square_km", "nhood_population"]]
nhoods_w_services = nhoods.merge(all_services, how="left", left_on="neighborhood", right_on="Район")
nhoods_w_services.drop(columns=["Район"], inplace=True)
nhoods_w_services.fillna(0, inplace=True)

In [13]:
# суммируем все услуги
# в идеале для каждой услуги отдельно рассчитывать по поправочным коэффициентам или нормированию

nhoods_w_services["sum_services"] = (nhoods_w_services["sports"] 
                                     + nhoods_w_services["adult_med"] 
                                     + nhoods_w_services["kids_med"]
                                     + nhoods_w_services["services"]
                                     + nhoods_w_services["trades"]
                                     + nhoods_w_services["eatpoint"])

In [14]:
# выводим плотность услуг, кол-во услуг на 1 кв. км
# выводим покрытие услуги, кол-во услуг на одного человека

nhoods_w_services["services_density"] = nhoods_w_services["sum_services"] / nhoods_w_services["nhood_area_in_square_km"]
nhoods_w_services["services_coverage"] = nhoods_w_services["sum_services"] / nhoods_w_services["nhood_population"]

In [15]:
nhoods_w_services

,administrative_district,neighborhood,nhood_area_in_square_km,nhood_population,sports,adult_med,kids_med,services,trades,eatpoint,sum_services,services_density,services_coverage
0,юго-западный,академический,583.0,110161,30.0,4.0,2.0,181.0,375.0,157.0,749.0,1.284734,0.006799
1,северо-восточный,алексеевский,529.0,79193,60.0,2.0,2.0,166.0,337.0,157.0,724.0,1.368620,0.009142
2,северо-восточный,алтуфьевский,325.0,57361,14.0,0.0,1.0,43.0,110.0,31.0,199.0,0.612308,0.003469
3,центральный,арбат,211.0,35796,19.0,1.0,0.0,138.0,442.0,301.0,901.0,4.270142,0.025170
4,северный,аэропорт,458.0,79283,36.0,2.0,0.0,217.0,447.0,204.0,906.0,1.978166,0.011427
...,...,...,...,...,...,...,...,...,...,...,...,...,...
141,новомосковский,сосенское,6707.0,39866,23.0,0.0,0.0,193.0,727.0,242.0,1185.0,0.176681,0.029725
142,троицкий,троицк,1633.0,65991,9.0,1.0,1.0,173.0,335.0,69.0,588.0,0.360073,0.008910
143,новомосковский,филимонковское,3577.0,8458,0.0,1.0,1.0,2.0,34.0,4.0,42.0,0.011742,0.004966
144,троицкий,щаповское,8606.0,10383,1.0,0.0,1.0,17.0,133.0,17.0,169.0,0.019637,0.016277


In [100]:
services_excel = nhoods_w_services[["administrative_district", "neighborhood", 
                                    "sum_services", "services_density", "services_coverage"]]
services_excel.to_excel(r"..\data\artefacts\services.xlsx")

### Наличие в районе объектов культуры, мест досуга и отдыха  ###

In [16]:
dosug_kidsplaygounds = pd.read_excel(r"..\data\data-mos-ru\data-dosug-kidsplaygounds.xlsx", skiprows=1)
dosug_kidsplaygounds = dosug_kidsplaygounds[["Административный округ", "Район", "geodata_center"]]

dosug_opensportsquares = pd.read_excel(r"..\data\data-mos-ru\data-dosug-opensportcomplex.xlsx", skiprows=1)
dosug_opensportsquares = dosug_opensportsquares[["Административный округ", "Район", "Адрес", 
                                                 "Форма посещения (платность)"]]

dosug_kidsparks = pd.read_excel(r"..\data\data-mos-ru\data-dosug-parks.xlsx", skiprows=1)
dosug_kidsparks = dosug_kidsparks[["Административный округ", "Район", "Адрес", "Форма посещения (платность)"]]

dosug_skateparks = pd.read_excel(r"..\data\data-mos-ru\data-dosug-skateparks.xlsx", skiprows=1)
dosug_skateparks = dosug_skateparks[["Административный округ", "Район", "Адрес"]]

In [17]:
kidsplaygounds = count_objects(dosug_kidsplaygounds, "kidsplaygounds")
opensportsquares = count_objects(dosug_opensportsquares, "opensportsquares")
kidsparks = count_objects(dosug_kidsparks, "kidsparks")
skateparks = count_objects(dosug_skateparks, "skateparks")

In [18]:
# объеденяем все фреймы в один

all_leisure = kidsplaygounds.merge(opensportsquares, how="outer")
all_leisure = all_leisure.merge(kidsparks, how="outer")
all_leisure = all_leisure.merge(skateparks, how="outer")
all_leisure.fillna(0, inplace=True)
all_leisure

,Район,kidsplaygounds_cnt,opensportsquares_cnt,kidsparks_cnt,skateparks_cnt
0,академический,25.0,72,1.0,0.0
1,алексеевский,1.0,38,1.0,0.0
2,алтуфьевский,10.0,31,0.0,0.0
3,арбат,18.0,9,0.0,0.0
4,аэропорт,6.0,37,3.0,1.0
...,...,...,...,...,...
141,троицк,0.0,20,1.0,1.0
142,щаповское,0.0,8,0.0,1.0
143,щербинка,0.0,23,0.0,0.0
144,южное медведково,0.0,29,2.0,3.0


In [19]:
# с вики-данных осатвляем нужные и соединяем их с фреймом кол-ва всех услуг

nhoods = neighborhoods[["administrative_district", "neighborhood", "nhood_area_in_square_km", "nhood_population"]]
nhoods_w_leisures = nhoods.merge(all_leisure, how="left", left_on="neighborhood", right_on="Район")
nhoods_w_leisures.drop(columns=["Район"], inplace=True)
nhoods_w_leisures.fillna(0, inplace=True)

In [20]:
# суммируем все услуги
# в идеале для каждой услуги отдельно рассчитывать по поправочным коэффициентам или нормированию

nhoods_w_leisures["sum_leisures"] = (nhoods_w_leisures["kidsplaygounds_cnt"] 
                                     + nhoods_w_leisures["opensportsquares_cnt"] 
                                     + nhoods_w_leisures["kidsparks_cnt"]
                                     + nhoods_w_leisures["skateparks_cnt"]
                                    )

In [21]:
# выводим плотность услуг, кол-во услуг на 1 кв. км
# выводим покрытие услуги, кол-во услуг на одного человека

nhoods_w_leisures["leisures_density"] = nhoods_w_leisures["sum_leisures"] / nhoods_w_leisures["nhood_area_in_square_km"]
nhoods_w_leisures["leisures_coverage"] = nhoods_w_leisures["sum_leisures"] / nhoods_w_leisures["nhood_population"]

In [22]:
nhoods_w_leisures

,administrative_district,neighborhood,nhood_area_in_square_km,nhood_population,kidsplaygounds_cnt,opensportsquares_cnt,kidsparks_cnt,skateparks_cnt,sum_leisures,leisures_density,leisures_coverage
0,юго-западный,академический,583.0,110161,25.0,72.0,1.0,0.0,98.0,0.168096,0.000890
1,северо-восточный,алексеевский,529.0,79193,1.0,38.0,1.0,0.0,40.0,0.075614,0.000505
2,северо-восточный,алтуфьевский,325.0,57361,10.0,31.0,0.0,0.0,41.0,0.126154,0.000715
3,центральный,арбат,211.0,35796,18.0,9.0,0.0,0.0,27.0,0.127962,0.000754
4,северный,аэропорт,458.0,79283,6.0,37.0,3.0,1.0,47.0,0.102620,0.000593
...,...,...,...,...,...,...,...,...,...,...,...
141,новомосковский,сосенское,6707.0,39866,1.0,43.0,0.0,3.0,47.0,0.007008,0.001179
142,троицкий,троицк,1633.0,65991,0.0,20.0,1.0,1.0,22.0,0.013472,0.000333
143,новомосковский,филимонковское,3577.0,8458,3.0,9.0,0.0,0.0,12.0,0.003355,0.001419
144,троицкий,щаповское,8606.0,10383,0.0,8.0,0.0,1.0,9.0,0.001046,0.000867


In [101]:
leisures_excel = nhoods_w_leisures[["administrative_district", "neighborhood", "sum_leisures",
                                    "leisures_density", "leisures_coverage"]]
leisures_excel.to_excel(r"..\data\artefacts\leisures.xlsx")

### возможность открытия бизнеса ###

In [23]:
# ЗАПРАШИВАЕМАЯ СТАВКА АРЕНДЫ И УРОВЕНЬ ВАКАНТНОСТИ ПО СУБРЫНКАМ, тыс. руб./м²/год и %

In [24]:
arenda_and_vacancy = nhoods.copy(deep=True)

arenda_and_vacancy["arenda_A"] = 0
arenda_and_vacancy["vacancy_lvl_A"] = 0

arenda_and_vacancy["arenda_B"] = 0
arenda_and_vacancy["vacancy_lvl_B"] = 0

In [25]:
#  Обзор рынка коммерческой недвижимости Москвы
# 1 полугодие 2022 преза, стр 20
class_A_central_arenda_mean = round(np.mean([35.7, 39.1, 26.4, 40.7, 43.3, 36.4, 38.9, 43.4]), 1)
class_A_central_vacancy_mean = round(np.mean([7,4,0,35,35,7,15,7]), 0)

In [26]:
# class A

arenda_and_vacancy.loc[(arenda_and_vacancy["administrative_district"] == "центральный"), "arenda_A"] = class_A_central_arenda_mean
arenda_and_vacancy.loc[(arenda_and_vacancy["administrative_district"] == "центральный"), "vacancy_lvl_A"] = class_A_central_vacancy_mean

arenda_and_vacancy.loc[(arenda_and_vacancy["administrative_district"] == "северо-восточный"), "arenda_A"] = 21.2
arenda_and_vacancy.loc[(arenda_and_vacancy["administrative_district"] == "северо-восточный"), "vacancy_lvl_A"] = 32.5

arenda_and_vacancy.loc[(arenda_and_vacancy["administrative_district"] == "северный"), "arenda_A"] = 35.9
arenda_and_vacancy.loc[(arenda_and_vacancy["administrative_district"] == "северный"), "vacancy_lvl_A"] = 7.5

arenda_and_vacancy.loc[(arenda_and_vacancy["administrative_district"] == "западный"), "arenda_A"] = 36.3
arenda_and_vacancy.loc[(arenda_and_vacancy["administrative_district"] == "западный"), "vacancy_lvl_A"] = 4

arenda_and_vacancy.loc[(arenda_and_vacancy["administrative_district"] == "северо-западный"), "arenda_A"] = 36.3
arenda_and_vacancy.loc[(arenda_and_vacancy["administrative_district"] == "северо-западный"), "vacancy_lvl_A"] = 4

arenda_and_vacancy.loc[(arenda_and_vacancy["administrative_district"] == "юго-западный"), "arenda_A"] = 20.4
arenda_and_vacancy.loc[(arenda_and_vacancy["administrative_district"] == "юго-западный"), "vacancy_lvl_A"] = 15

arenda_and_vacancy.loc[(arenda_and_vacancy["administrative_district"] == "восточный"), "arenda_A"] = 14.2
arenda_and_vacancy.loc[(arenda_and_vacancy["administrative_district"] == "восточный"), "vacancy_lvl_A"] = 32.5

arenda_and_vacancy.loc[(arenda_and_vacancy["administrative_district"] == "юго-восточный"), "arenda_A"] = 17 # assumption, mean of nearest nhoods
arenda_and_vacancy.loc[(arenda_and_vacancy["administrative_district"] == "юго-восточный"), "vacancy_lvl_A"] = 20 # assumption, mean of nearest nhoods

arenda_and_vacancy.loc[(arenda_and_vacancy["administrative_district"] == "южный"), "arenda_A"] = 17 # assumption, mean of nearest nhoods
arenda_and_vacancy.loc[(arenda_and_vacancy["administrative_district"] == "южный"), "vacancy_lvl_A"] = 20 # assumption, mean of nearest nhoods


In [27]:
# 1 полугодие 2022 преза, стр 20
class_B_central_arenda_mean = round(np.mean([18.8, 25.8, 23.5, 16.5, 29.7, 18.5, 27.2, 19.7, 31.5,31.7]), 1)
class_B_central_vacancy_mean = round(np.mean([16,13,11,9,7,5,5,5,7]), 0)

In [28]:
# class B

arenda_and_vacancy.loc[(arenda_and_vacancy["administrative_district"] == "центральный"), "arenda_B"] = class_B_central_arenda_mean
arenda_and_vacancy.loc[(arenda_and_vacancy["administrative_district"] == "центральный"), "vacancy_lvl_B"] = class_B_central_vacancy_mean

arenda_and_vacancy.loc[(arenda_and_vacancy["administrative_district"] == "северо-восточный"), "arenda_B"] = 15
arenda_and_vacancy.loc[(arenda_and_vacancy["administrative_district"] == "северо-восточный"), "vacancy_lvl_B"] = 7

arenda_and_vacancy.loc[(arenda_and_vacancy["administrative_district"] == "северный"), "arenda_B"] = 16.79
arenda_and_vacancy.loc[(arenda_and_vacancy["administrative_district"] == "северный"), "vacancy_lvl_B"] = 9

arenda_and_vacancy.loc[(arenda_and_vacancy["administrative_district"] == "западный"), "arenda_B"] = 17.1
arenda_and_vacancy.loc[(arenda_and_vacancy["administrative_district"] == "западный"), "vacancy_lvl_B"] = 9

arenda_and_vacancy.loc[(arenda_and_vacancy["administrative_district"] == "северо-западный"), "arenda_B"] = 17.1
arenda_and_vacancy.loc[(arenda_and_vacancy["administrative_district"] == "северо-западный"), "vacancy_lvl_B"] = 9

arenda_and_vacancy.loc[(arenda_and_vacancy["administrative_district"] == "юго-западный"), "arenda_B"] = 15.7
arenda_and_vacancy.loc[(arenda_and_vacancy["administrative_district"] == "юго-западный"), "vacancy_lvl_B"] = 9

arenda_and_vacancy.loc[(arenda_and_vacancy["administrative_district"] == "восточный"), "arenda_B"] = 12.7
arenda_and_vacancy.loc[(arenda_and_vacancy["administrative_district"] == "восточный"), "vacancy_lvl_B"] = 5

arenda_and_vacancy.loc[(arenda_and_vacancy["administrative_district"] == "юго-восточный"), "arenda_B"] = 18.9
arenda_and_vacancy.loc[(arenda_and_vacancy["administrative_district"] == "юго-восточный"), "vacancy_lvl_B"] = 5

arenda_and_vacancy.loc[(arenda_and_vacancy["administrative_district"] == "южный"), "arenda_B"] = 18.9 
arenda_and_vacancy.loc[(arenda_and_vacancy["administrative_district"] == "южный"), "vacancy_lvl_B"] = 5


In [29]:
arenda_and_vacancy = arenda_and_vacancy[arenda_and_vacancy["arenda_A"] != 0]

In [32]:
# building_permissions, Январь-Июолб 2022, стр 9

arenda_and_vacancy.loc[(arenda_and_vacancy["administrative_district"] == "центральный"), "building_permissions"] = 106.7
arenda_and_vacancy.loc[(arenda_and_vacancy["administrative_district"] == "северо-восточный"), "building_permissions"] = 352.5
arenda_and_vacancy.loc[(arenda_and_vacancy["administrative_district"] == "северный"), "building_permissions"] = 321.5
arenda_and_vacancy.loc[(arenda_and_vacancy["administrative_district"] == "западный"), "building_permissions"] = 628.4
arenda_and_vacancy.loc[(arenda_and_vacancy["administrative_district"] == "северо-западный"), "building_permissions"] = 200.3
arenda_and_vacancy.loc[(arenda_and_vacancy["administrative_district"] == "юго-западный"), "building_permissions"] = 94.7
arenda_and_vacancy.loc[(arenda_and_vacancy["administrative_district"] == "восточный"), "building_permissions"] = 393.3
arenda_and_vacancy.loc[(arenda_and_vacancy["administrative_district"] == "юго-восточный"), "building_permissions"] = 558.1
arenda_and_vacancy.loc[(arenda_and_vacancy["administrative_district"] == "южный"), "building_permissions"] = 389

In [82]:
arenda_and_vacancy_excel = arenda_and_vacancy[["administrative_district", "neighborhood", 
                                               "arenda_A", "vacancy_lvl_A", "arenda_B", "vacancy_lvl_B", 
                                               "building_permissions"]]
arenda_and_vacancy_excel.to_excel(r"..\data\artefacts\arenda_and_vacancy.xlsx")

### Прибыль по районам и кол-во ЮЛ и ИП ###

In [96]:
district_income = pd.read_csv(r"..\data\spark\spark_msk_income.txt", sep="\t")
district_income = preprocess(district_income, "Субъект федерации")
district_income["Средняя прибыль,млн рублей"] = district_income["Средняя прибыль,млн рублей"].str.replace(",", ".").astype(float)
district_income["Общая прибыль,млн рублей"] = district_income["Общая прибыль,млн рублей"].str.replace(",", ".")
district_income["Общая прибыль,млн рублей"] = district_income["Общая прибыль,млн рублей"].str.replace(" ", "").astype(float)

In [123]:
district_company_count = pd.read_csv(r"..\data\spark\spark_msk_company_cnt.txt", sep="\t")

district_company_count = district_company_count.reset_index()
district_company_count.rename(columns={"level_1": "Субъект федерации", 
                      "Субъект федерации": "Количество компаний", 
                      "Количество компаний": "Доля"},
                inplace=True)
district_company_count.drop(columns=["level_0", "perc"], inplace=True)
district_company_count = preprocess(district_company_count, "Субъект федерации")
district_company_count["Количество компаний"] = district_company_count["Количество компаний"].str.replace(" ", "").astype(int)
district_company_count.drop_duplicates(subset=["Субъект федерации"], inplace=True)

In [98]:
district_income_excel = district_income[["Субъект федерации", "Средняя прибыль,млн рублей", "Общая прибыль,млн рублей"]]
district_income_excel.to_excel(r"..\data\artefacts\district_income.xlsx")

In [124]:
district_company_count_excel = district_company_count[["Субъект федерации", "Количество компаний"]]
district_company_count_excel.to_excel(r"..\data\artefacts\district_company_count.xlsx")

In [37]:
# можем посмотреть районы с низкими арендными ставками, большим кол-вом вакантных площадей, большой плотностью населения,
# кол-вом готовящегося к сдаче жилья. Из этого сделать вывод о перспективах открытия бизнеса 
# интуиция: можно дешево взять в аренду помещение, много людей, будет еще больше

# видим перспективы бизнеса по районам

### ЖКХ и социалка ###

In [38]:
soc_UK = pd.read_excel(r"..\data\data-mos-ru\data-soc-UK.xlsx", skiprows=1)
soc_avr = pd.read_excel(r"..\data\data-mos-ru\data-soc-avariinie-sluzbi.xlsx")
soc_MFC = pd.read_excel(r"..\data\data-mos-ru\data-soc-MFC.xlsx", skiprows=1)
soc_center = pd.read_excel(r"..\data\data-mos-ru\data-soc-center.xlsx", skiprows=1)

In [39]:
# управляющие компании

part1 = soc_UK[soc_UK["Адреса мест приема граждан"].notnull()] # выдрать округ и район регулярками
part2 = soc_UK[soc_UK["Адреса мест приема граждан"].isnull()] # выдрать округ и район через геоцентр

part1["Административный округ"] = part1["Адреса мест приема граждан"].str.extract(r"(AdmArea:[А-яа-яё -]*)")
part1["Район"] = part1["Адреса мест приема граждан"].str.extract(r"(District:[А-яа-яё -]*)")

part2 = transform_from_geocenter(part2, gdf)
part2.drop(columns=["index_left", "cell_zid", "sub_ter", "boundary"], inplace=True)

soc_UK = part1.append(part2) # соединить обратно датафреймы

soc_UK = preprocess(soc_UK, "Административный округ")
soc_UK = preprocess(soc_UK, "Район")

UK = count_objects(soc_UK, "управляющие_компании")
UK = UK[UK["Район"] != ""]

In [40]:
# аварийные службы

soc_avr.rename(columns={"AdmArea":"Административный округ", "District":"Район"}, inplace=True)
soc_avr = preprocess(soc_avr, "Административный округ")
soc_avr = preprocess(soc_avr, "Район")

avr = count_objects(soc_avr, "аварийки")

In [41]:
# МФЦ по районам

soc_MFC.rename(columns={"Административный округ по адресу":"Административный округ"}, inplace=True)
soc_MFC = preprocess(soc_MFC, "Район")

MFC = count_objects(soc_MFC, "МФЦ")

In [42]:
# социальные центры
soc_center["Административный округ"] = soc_center["Адреса объекта"].str.extract(r"(AdmArea:[А-яа-яё -]*)")
soc_center["Район"] = soc_center["Адреса объекта"].str.extract(r"(District:[А-яа-яё -]*)")
soc_center = preprocess(soc_center, "Административный округ")
soc_center = preprocess(soc_center, "Район")

centers = count_objects(soc_center, "соц_центры")

In [43]:
# мержим все соц услуги в один фрейм

all_social = UK.merge(avr, how="outer")
all_social = all_social.merge(MFC, how="outer")
all_social = all_social.merge(centers, how="outer")

In [44]:
# приляпливаем фрейм с услугами к районам

social = nhoods.copy(deep=True)
social = social.merge(all_social, how="outer", left_on="neighborhood", right_on="Район")
social.drop(columns=["Район"], inplace=True)
social = social[social["neighborhood"].notnull()]
social.fillna(0, inplace=True)

In [45]:
# суммируем все услуги
# в идеале для каждой услуги отдельно рассчитывать по поправочным коэффициентам или нормированию

social["sum_socials"] = (social["управляющие_компании_cnt"] 
                         + social["аварийки_cnt"] 
                         + social["МФЦ_cnt"]
                         + social["соц_центры_cnt"]
                         )

In [46]:
# выводим плотность услуг, кол-во услуг на 1 кв. км
# выводим покрытие услуги, кол-во услуг на одного человека

social["socials_density"] = social["sum_socials"] / social["nhood_area_in_square_km"]
social["socials_coverage"] = social["sum_socials"] / social["nhood_population"]

In [102]:
social_excel = social[["administrative_district", "neighborhood", "socials_density", "sum_socials", "socials_coverage"]]
social_excel.to_excel(r"..\data\artefacts\social.xlsx")

### образование ###

In [48]:
# школы, колледжи, детсады, кроме вузов!
edu_gosedu = pd.read_excel(r"..\data\data-mos-ru\data-edu-gosedu.xlsx", skiprows=1)
edu_gosedu = edu_gosedu[["Статус организации", "Юридический адрес", "geodata_center"]]

edu_dopeducation = pd.read_excel(r"..\data\data-mos-ru\data-edu-dopeducation.xlsx", skiprows=1)
edu_dopeducation = edu_dopeducation[["Административный округ по адресу", "Район", "Юридический адрес"]]
edu_dopeducation.rename(columns={"Административный округ по адресу": "Административный округ"}, inplace=True)

# а вот и ВУЗЫ
# http://edu.repetitor-general.ru/maps/vuzy.php
university = pd.read_csv(r"..\data\university\university.txt", sep="\t")
university = university[["Учебное заведение", "Район"]]

In [49]:
gosedu = transform_from_geocenter(edu_gosedu, gdf)
dopedu = preprocess(edu_dopeducation, "Район")

In [50]:
# препроцесс файла с вузами. один вуз может быть расположен в разных районах, поэтому немного заморочено

expanded_districts = university["Район"].str.split(" ", expand=True)
uni = university[["Учебное заведение"]].join(expanded_districts)
uni = pd.DataFrame(uni.melt(id_vars=["Учебное заведение"]))

uni.dropna(inplace=True)
uni.drop(columns=["variable"], inplace=True)

uni.loc[uni.value.str.contains("Марьина"), "value"] = "Марьина роща"
uni.loc[uni.value.str.contains("роща"), "value"] = "Марьина роща"
uni.loc[uni.value.str.contains("Тёплый"), "value"] = "Тёплый стан"
uni.loc[uni.value.str.contains("Стан"), "value"] = "Тёплый стан"

uni.drop_duplicates(inplace=True)
uni.rename(columns={"value": "Район"}, inplace=True)
uni = preprocess(uni, "Район")

adm_distr = nhoods[["administrative_district", "neighborhood"]]
uni = uni.merge(adm_distr, how="left", left_on="Район", right_on="neighborhood")
uni.rename(columns={"administrative_district": "Административный округ"}, inplace=True)

In [51]:
gosedu_grp = count_objects(gosedu, "гос_обр")
dopedu_grp = count_objects(dopedu, "доп_обр")
uni_grp = count_objects(uni, "ВУЗ")

In [52]:
all_edu = gosedu_grp.merge(dopedu_grp, how="outer")
all_edu = all_edu.merge(uni_grp, how="outer")

In [53]:
edu = nhoods.merge(all_edu, how="left", left_on="neighborhood", right_on="Район")
edu.drop(columns=["Район"], inplace=True)
edu = edu[edu["neighborhood"].notnull()]
edu.fillna(0, inplace=True)

In [54]:
edu["sum_edu"] = edu["гос_обр_cnt"] + edu["доп_обр_cnt"] + edu["ВУЗ_cnt"]

In [55]:
# выводим плотность услуг, кол-во услуг на 1 кв. км
# выводим покрытие услуги, кол-во услуг на одного человека

edu["edu_density"] = edu["sum_edu"] / edu["nhood_area_in_square_km"]
edu["edu_coverage"] = edu["sum_edu"] / edu["nhood_population"]

In [103]:
edu_excel = edu[["administrative_district", "neighborhood", "edu_density", "sum_edu", "edu_coverage"]]
edu_excel.to_excel(r"..\data\artefacts\edu.xlsx")

### МЕТРО ###

In [112]:
metro_geo = pd.read_excel(r"..\data\data-mos-ru\data-metro-geo.xlsx", skiprows=1)
metro_geo = metro_geo[["Административный округ", "Район", "Станция метрополитена"]]

metro_traffic = pd.read_excel(r"..\data\data-mos-ru\data-metro-traffic.xlsx", skiprows=1)
metro_traffic = metro_traffic[["Станция метрополитена", "Квартал", "Входы пассажиров", "Выходы пассажиров"]]

In [113]:
metro_geo.drop_duplicates(inplace=True)

metro_geo = preprocess(metro_geo, "Район")
metro_geo = preprocess(metro_geo, "Административный округ")
metro_geo = preprocess(metro_geo, "Станция метрополитена")

In [114]:
# делим квартальный трафик на 3 для понимания трафика в месяц

metro_traffic["Входы пассажиров в мес"] = round(metro_traffic["Входы пассажиров"] / 3, 0)
metro_traffic["Выходы пассажиров в мес"] = round(metro_traffic["Выходы пассажиров"] / 3, 0)

In [115]:
# группируем и берем медиану месяцов разных кварталов
# соединяем даатсеты, удаляем инфу по районам для которых у нас нет трафика

grouped_median_mt = metro_traffic.groupby(["Станция метрополитена"], as_index=False).agg("median")
grouped_median_mt = preprocess(grouped_median_mt, "Станция метрополитена")

metro = metro_geo.merge(grouped_median_mt[["Станция метрополитена", "Входы пассажиров в мес", "Выходы пассажиров в мес"]], 
                        how="left")

metro = metro[metro["Входы пассажиров в мес"] != 0]

# группируем кол-во входов/выходов по районам
metro = metro.drop(columns=["Станция метрополитена"])
metro = metro.groupby(["Район"], as_index=False).agg(sum)

In [118]:
metro_excel = metro[["Район", "Входы пассажиров в мес", "Выходы пассажиров в мес"]]
metro_excel.to_excel(r"..\data\artefacts\metro_traffic.xlsx")